In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
import torchvision.transforms as transforms

import torchvision

import numpy as np

In [ ]:
# ResNet
"""
  skip-connection :
    y = f(x) + x
  
  1x1 convolution의 역할 :
    channel의 축소
    ex ) 28 * 28 * 192  conv  1 * 1 * 16
      => 28 * 28 * 16으로 데이터 줄이기 가능

"""
class ResidualNet(nn.Module):
  def __init__(self, output_dim):
    super(ResidualNet, self).__init__()

    self.n_classes = output_dim

    self.seq1 = nn.Sequential(
        nn.Conv2d(1, 32, stride = 2, kernel_size = 2),
        nn.ReLU()
        nn.Conv2d(32, 64, stride = 2, kernel_size = 2),
        nn.ReLU()
    )
    self.seq2 = nn.Sequential(

    )

    self.softmax = nn.Softmax(dim = -1)
    self.fc = nn.Linear(128 * 3 * 3 * 16,self.n_classes)

  def forward(self,x):
    identity = x
    out = self.seq1(x)
    out = self.seq2(out)

    #flatten
    out = out.view(x.size(0),-1)
    out = self.softmax(torch.argmax(out))
    return out

In [ ]:
mnist = torchvision.datasets.MNIST('./',download = True,
                                          transform = transforms.ToTensor())
data_loader = torch.utils.data.DataLoader(mnist,
                                          batch_size = 5,
                                          shuffle = True
                                          )


  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw



In [ ]:
dataset_size = len(mnist)
train_size = int(dataset_size * 0.8)
valid_size = int(dataset_size * 0.1)
test_size = dataset_size - train_size - valid_size
train_dl, valid_dl, test_dl = random_split(mnist,[train_size, valid_size, test_size])

print(f'train data size : {len(train_dl)}')
print(f'valid data size : {len(valid_dl)}')
print(f'test data size : {len(test_dl)}')

train data size : 48000
valid data size : 6000
test data size : 6000


In [ ]:
train_loader = DataLoader(train_dl, batch_size = 16, shuffle = True, drop_last = True)
valid_loader = DataLoader(valid_dl, batch_size = 16, shuffle = True, drop_last = True)
test_loader = DataLoader(test_dl, batch_size = 16, shuffle = True, drop_last = True)

device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
model = ResidualNet(10).to(device)
criterion = torch.nn.Functional.CrossEntorpyLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.001)

AttributeError: ignored

In [ ]:
for epoch in range(1001):
  cost = 0
  for x, y in train_loader:
    x = x.to(device)
    y = y.to(device)
    print (x.shape)
    pred = model(x)
    loss = criterion(y, pred)
    optimizer.zero_grad()

    loss.backward()
    optimizer.step()

    cost += loss
  
  cost = cost / len(train_loader)
  
  if epoch % 100 == 0:
    print(f"Epoch {epoch} : {cost}")

with torch.no_grad():
  model.eval()

  cost = 0
  for x,y in valid_loader:
    x = x.to(device)
    y = y.to(device)

    pred = model(x)
    loss = criterion(pred, y)

    cost += loss

  cost /= len(valid_loader)
  print(f"Acc : {cost}")



torch.Size([16, 1, 28, 28])


RuntimeError: ignored